In [ ]:
import kaggle

In [ ]:
#The standard list of emotions we would be using 
REG_EMOTION_LIST = ['fear', 'sadness', 'anger', 'disgust', 'joy', 'surprise']

**THE FIRST DATASET SOURCE**

In [ ]:
!kaggle datasets download -d 'pashupatigupta/emotion-detection-from-text'

  0% 0.00/1.56M [00:00<?, ?B/s]
100% 1.56M/1.56M [00:00<00:00, 124MB/s]


In [ ]:
import zipfile

In [ ]:
zip_ref = zipfile.ZipFile('/content/emotion-detection-from-text.zip')
zip_ref.extractall('/content/emotion-detection-from-text')
zip_ref.close()

In [ ]:
rm /content/emotion-detection-from-text.zip

In [ ]:
import pandas as pd

tweet_emotions = pd.read_csv('/content/emotion-detection-from-text/tweet_emotions.csv')

In [ ]:
tweet_emotions.tail(7)

,tweet_id,sentiment,content
39993,1753918892,neutral,"@sendsome2me haha, yeah. Twitter has many uses..."
39994,1753918900,happiness,Succesfully following Tayla!!
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...
39999,1753919049,love,@mopedronin bullet train from tokyo the gf ...


In [ ]:
tweet_emotions.drop(axis=1, labels='tweet_id', inplace=True)

In [ ]:
!pip install emoji
from emoji import demojize

#change all to lowercase
tweet_emotions['content'] = tweet_emotions['content'].str.lower()

#remove user names @[NAME] and links
tweet_emotions['content'] = tweet_emotions['content'].str.replace(r'(http|@)\S*\s*', '')

#turn emojis to words
tweet_emotions['content'] = tweet_emotions['content'].apply(demojize)

#remove non ascii and special characters
tweet_emotions['content'] = tweet_emotions['content'].str.replace(r'[^a-z\':_]',' ')

#remove multiple spaces - though it isn't important
tweet_emotions['content'] = tweet_emotions['content'].str.replace(r'\s+',' ')

#remove empty contents
tweet_emotions.drop(index=(tweet_emotions[tweet_emotions['content']=='']).index, inplace=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=45f144de5e46d91d153cc1709791d317688ff9e01e154f1f776162e5b24f31b8
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.


In [ ]:
#import nltk

#nltk.download('stopwords')
#stopwords = nltk.corpus.stopwords.words('english')
#stopwords.remove('no')
#stopwords.remove('nor')
#stopwords.remove('not')
#stopwords.remove('won')

#remove unecessary stopwords
#tweet_emotions['content'] = tweet_emotions['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))


In [ ]:
tweet_emotions.groupby(['sentiment']).count()

,content
sentiment,
anger,110
boredom,179
empty,816
enthusiasm,759
fun,1776
happiness,5208
hate,1323
love,3841
neutral,8581


In [ ]:
sentiment_groups = tweet_emotions.groupby('sentiment').groups

#removing the examples of the labels boredom, empty, neutral, worry
sentiment_idx = sentiment_groups['boredom'].to_list() + sentiment_groups['empty'].to_list() + sentiment_groups['neutral'].to_list() + sentiment_groups['worry'].to_list()

tweet_emotions.drop(index=sentiment_idx, inplace=True, )

In [ ]:
tweet_emotions.groupby('sentiment').count()

,content
sentiment,
anger,110
enthusiasm,759
fun,1776
happiness,5208
hate,1323
love,3841
relief,1526
sadness,5162
surprise,2187


In [ ]:
emo_transform_dict = {'sadness':'sadness', 'happiness':'joy','hate':'anger','relief':'joy',
                  'anger':'anger', 'enthusiasm':'joy', 'love':'joy', 'fun':'joy','surprise':'surprise'}

def transform_func(emotion):
  return emo_transform_dict[emotion]

tweet_emotions['sentiment'] = tweet_emotions['sentiment'].apply(transform_func)

In [ ]:
#the joy label in the dataset is too much so to balance out a bit we will remove all the joy labels from this example
tweet_emotions.drop(index=tweet_emotions.groupby('sentiment').groups['joy'], inplace=True)

In [ ]:
tweet_emotions.groupby('sentiment').count()

,content
sentiment,
anger,1433
sadness,5162
surprise,2187


**THE SECOND DATASET SOURCE**

In [ ]:
!kaggle datasets download -d chandrug/textemotiondetection

  0% 0.00/721k [00:00<?, ?B/s]
100% 721k/721k [00:00<00:00, 104MB/s]


In [ ]:
zip_ref2 = zipfile.ZipFile('/content/textemotiondetection.zip')
zip_ref2.extractall('/content/textemotiondetection')
zip_ref2.close()

In [ ]:
!rm /content/textemotiondetection.zip

In [ ]:
with open('/content/textemotiondetection/train.txt', 'r') as f:
  emotion_train = f.read()

with open('/content/textemotiondetection/test.txt', 'r') as f:
  emotion_test = f.read()

with open('/content/textemotiondetection/val.txt', 'r') as f:
  emotion_val = f.read()

In [ ]:
def preprocess_to_df(string):
  list_of_strings = string.split('\n')
  prepared_data = [s.split(';') for s in list_of_strings]
  return pd.DataFrame(prepared_data, columns=['content', 'sentiment'])

In [ ]:
emotion_train = preprocess_to_df(emotion_train)

emotion_test = preprocess_to_df(emotion_test)

emotion_val = preprocess_to_df(emotion_val)


emotion_train.drop(axis=0, inplace=True, index=emotion_train.index[-1])

emotion_test.drop(axis=0, inplace=True, index=emotion_test.index[-1])

emotion_val.drop(axis=0, inplace=True, index=emotion_val.index[-1])


emotion_data = pd.concat([emotion_train , emotion_test, emotion_val])

In [ ]:
#The sentences with 'a http href' are hard to understand
emotion_data.drop(index=(emotion_data[emotion_data['content'].str.contains(r'(http|href)')].index), inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [ ]:
REG_EMOTION_LIST = ['fear', 'sadness', 'anger', 'disgust', 'joy', 'surprise']

In [ ]:
emotion_data.groupby('sentiment').count()

,content
sentiment,
anger,2672
fear,2344
joy,6621
love,1598
sadness,5670
surprise,707


In [ ]:
emo_transform_dict2 = {'anger':'anger', 'fear':'fear', 'sadness':'sadness', 'joy':'joy', 'love':'joy', 'surprise':'surprise'}

def transform_func(emotion):
  return emo_transform_dict2[emotion]

emotion_data['sentiment'] = emotion_data['sentiment'].apply(transform_func)

In [ ]:
emotion_data.groupby('sentiment').count()

,content
sentiment,
anger,2672
fear,2344
joy,8219
sadness,5670
surprise,707


**THE THIRD DATASET SOURCE**

In [ ]:
!wget -P goemotions https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv

!wget -P goemotions https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv

!wget -P goemotions https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/dev.tsv

--2022-07-03 05:14:56--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3519053 (3.4M) [text/plain]
Saving to: ‘goemotions/train.tsv’

train.tsv           100%[===================>]   3.36M  --.-KB/s    in 0.07s   

2022-07-03 05:14:56 (45.7 MB/s) - ‘goemotions/train.tsv’ saved [3519053/3519053]

--2022-07-03 05:14:56--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awa

In [ ]:
goemotion_train = pd.read_csv('/content/goemotions/train.tsv', sep='\t', header=None)

goemotion_test = pd.read_csv('/content/goemotions/test.tsv', sep='\t', header=None)

goemotion_dev = pd.read_csv('/content/goemotions/dev.tsv', sep='\t', header=None)


goemotion_train.columns = ['content', 'sentiment', 'id']

goemotion_test.columns = ['content', 'sentiment', 'id']

goemotion_dev.columns = ['content', 'sentiment', 'id']


goemotion_data = pd.concat([goemotion_train , goemotion_test, goemotion_dev], ignore_index=True)

goemotion_data.drop(axis=1, labels='id', inplace=True)

In [ ]:
!pip install emoji
from emoji import demojize

#change all to lowercase
goemotion_data['content'] = goemotion_data['content'].str.lower()

#turn emojis to words
goemotion_data['content'] = goemotion_data['content'].apply(demojize)

#remove non ascii and special characters
goemotion_data['content'] = goemotion_data['content'].str.replace(r'[^a-z\':_]',' ')

#remove multiple spaces - though it isn't important
goemotion_data['content'] = goemotion_data['content'].str.replace(r'\s+',' ')

#remove empty contents
goemotion_data.drop(index=(goemotion_data[goemotion_data['content']=='']).index, inplace=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
emotion_list = ['admiration', 'amusement', 'anger', 'annoyance', 'approval',
                'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
                'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
                'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
                'pride', 'realization', 'relief', 'remorse', 'sadness','surprise',
                'neutral']

ekman_mapping = {"anger": ["anger", "annoyance", "disapproval"],
"disgust": ["disgust"],
"fear": ["fear", "nervousness"],
"joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"],
"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
"surprise": ["surprise", "realization", "confusion", "curiosity"]
}

#unlike the others due to the label type of our output being numbers it'll map from numbers to string
emo_transform_dict3 = {}

for key, vals in ekman_mapping.items():
    for val in vals:
      emo_transform_dict3[str(emotion_list.index(val))] = key

emo_transform_dict3['27'] = 'neutral'

In [ ]:
emo_transform_dict3

{'0': 'joy',
 '1': 'joy',
 '10': 'anger',
 '11': 'disgust',
 '12': 'sadness',
 '13': 'joy',
 '14': 'fear',
 '15': 'joy',
 '16': 'sadness',
 '17': 'joy',
 '18': 'joy',
 '19': 'fear',
 '2': 'anger',
 '20': 'joy',
 '21': 'joy',
 '22': 'surprise',
 '23': 'joy',
 '24': 'sadness',
 '25': 'sadness',
 '26': 'surprise',
 '27': 'neutral',
 '3': 'anger',
 '4': 'joy',
 '5': 'joy',
 '6': 'surprise',
 '7': 'surprise',
 '8': 'joy',
 '9': 'sadness'}

In [ ]:
def transform_func(emotion):
    emotion = emotion.split(',')
    emotion = [emo_transform_dict3[emo] for emo in emotion]
    return ','.join(set(emotion))

In [ ]:
goemotion_data['sentiment'] = goemotion_data['sentiment'].apply(transform_func)

In [ ]:
goemotion_data[['first', 'second', 'third']] = goemotion_data['sentiment'].str.split(pat=',', expand=True)

In [ ]:
first_opinion = goemotion_data[goemotion_data['first'].notna()][['content', 'first']]

first_opinion.columns = ['content', 'sentiment']

second_opinion = goemotion_data[goemotion_data['second'].notna()][['content', 'second']]

second_opinion.columns = ['content', 'sentiment']

third_opinion = goemotion_data[goemotion_data['third'].notna()][['content', 'third']]

third_opinion.columns = ['content', 'sentiment']

In [ ]:
goemotion_data = pd.concat([first_opinion, second_opinion, third_opinion], ignore_index=True, )

In [ ]:
goemotion_data

,content,sentiment
0,my favourite food is anything i didn't have to cook myself,neutral
1,now if he does off himself everyone will think hes having a laugh screwing with people instead of actually dead,neutral
2,why the fuck is bayless isoing,anger
3,to make her feel threatened,fear
4,dirty southern wankers,anger
...,...,...
59164,i owe nothing and they take more than that every year,anger
59165,never made it through it makes me physically ill just seeing images of that monster combined with him describing his incomprehensible evil,disgust
59166,deep blue eyes look weird to me the rest seems pretty good,disgust
59167,bless this woman that honestly sounds horrific from every end i hope her ex name has lived a horrendous life since perpetuating that disgusting act,disgust


In [ ]:
goemotion_data.drop(index=goemotion_data[goemotion_data['sentiment']=='neutral'].index, inplace=True)

In [ ]:
goemotion_data.groupby('sentiment').count()

,content
sentiment,
anger,7022
disgust,1013
fear,929
joy,21733
sadness,4032
surprise,6668


**COMBINING THE DATASETS**

In [ ]:
final_data = pd.concat([tweet_emotions, emotion_data, goemotion_data], ignore_index=True)

In [ ]:
final_data.groupby('sentiment').count()

,content
sentiment,
anger,11127
disgust,1013
fear,3273
joy,29952
sadness,14864
surprise,9562


In [ ]:
#save the data
final_data.to_csv('sentiment-analysis-data.csv', index=False)